# Data Cleaning and Processing

In [1]:
import os
import re
import sys
import numpy as np
import pandas as pd

In [2]:
print(sys.version)

3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:27:44) [MSC v.1900 64 bit (AMD64)]


In [3]:
data = pd.read_csv(os.getcwd() + '/data/nuforc_reports_07202018.csv', sep=',') #import scraped results

In [4]:
data.head()

,Occurred,Reported,Posted,Location,Shape,Duration,Report
0,6/24/1996 00:30,6/26/1996 12:42,11/2/1999,"Aurora, CO",changed,1 hour,Referred to Marilyn Childs to call CO state di...
1,7/17/2018 03:20 (Entered as: 07/17/18 3:20),7/17/2018 3:07:02 AM 03:07,7/19/2018,"Phoenix, AZ",NaN,1 minute,Silent red lights float over interstate 17Whil...
2,5/15/2018 21:30 (Entered as: 05/16/18 09:30),5/16/2018 12:57:43 AM 00:57,5/31/2018,"Reno, NV",NaN,20 minutes,"Hi, I’m a huge skeptic who wants to believe, a..."
3,5/20/2018 13:00 (Entered as: 05/20/2018 13:00),5/23/2018 5:32:16 AM 05:32,5/31/2018,"Johnstown, PA",NaN,10 seconds,My name is ((name deleted)) and I live in Camb...
4,5/20/2018 13:10 (Entered as: 05/20/18 13:10),6/1/2018 2:06:25 PM 14:06,6/7/2018,"Helena, MT",NaN,NaN,MADAR Node 128


## Dealing with null values

In [5]:
NULL = 100 * (data.shape[0] - data.describe().T['count']) / data.shape[0]; # percent of null values in each column
NULL

Occurred      0.0439303
Reported              0
Posted                0
Location       0.114563
Shape           3.18193
Duration        3.38522
Report      0.000861378
Name: count, dtype: object

In [6]:
data = data.dropna() #drop null values (roughly 3.4% of the original row count)
data = data.reset_index(drop=True) #reset index values

## Dealing with messy datetimes

In [7]:
data.head()

,Occurred,Reported,Posted,Location,Shape,Duration,Report
0,6/24/1996 00:30,6/26/1996 12:42,11/2/1999,"Aurora, CO",changed,1 hour,Referred to Marilyn Childs to call CO state di...
1,(Entered as: unknown),3/21/2006 12:11:10 PM 12:11,5/15/2006,"Bristol, PA",Light,About 1 min.,"Hi, I am an avid sky watcher and usually I not..."
2,00:45 (Entered as: 05# 0:45),5/16/2015 12:34:36 AM 00:34,5/22/2015,"Saskatoon (Canada), SK",Light,2 minutes,We seemed to know to look up and and for me it...
3,01:00 (Entered as: 01:00),10/21/2001 11:50:39 AM 11:50,11/20/2001,Dublin (Republic of Ireland),Light,a few seconds,"A ball of light, lighting up my pitch dark roo..."
4,01:00 (Entered as: Continious fall \'98-sprin...,9/2/1999 09:52,2/5/2003,"Careywood & Bayview (Pend Oreille Lake), ID",Light,few minutes,"Ball of light similiar to reported ""Earht Ligh..."


The occurred column can be problematic. Reported and posted dates and times are well formatted.

In [8]:
pattern = '(\d+\/\d+\/\d+ \d+:\d+)|(\d+\/\d+\/\d+)|(\d+:\d+)|(\d+ \d+)|\(Entered as:(.*?)\)' #grab datetimes
data['Occurred'] = data['Occurred'].apply(lambda x: list(filter(None,re.findall(pattern, str(x))[0]))[0]) #filter

In [9]:
data.head()

,Occurred,Reported,Posted,Location,Shape,Duration,Report
0,6/24/1996 00:30,6/26/1996 12:42,11/2/1999,"Aurora, CO",changed,1 hour,Referred to Marilyn Childs to call CO state di...
1,unknown,3/21/2006 12:11:10 PM 12:11,5/15/2006,"Bristol, PA",Light,About 1 min.,"Hi, I am an avid sky watcher and usually I not..."
2,00:45,5/16/2015 12:34:36 AM 00:34,5/22/2015,"Saskatoon (Canada), SK",Light,2 minutes,We seemed to know to look up and and for me it...
3,01:00,10/21/2001 11:50:39 AM 11:50,11/20/2001,Dublin (Republic of Ireland),Light,a few seconds,"A ball of light, lighting up my pitch dark roo..."
4,01:00,9/2/1999 09:52,2/5/2003,"Careywood & Bayview (Pend Oreille Lake), ID",Light,few minutes,"Ball of light similiar to reported ""Earht Ligh..."


In [10]:
def time_picker(series):
    #PICK OUT TIME STRINGS FROM DATETIMES
    times = {}
    for t in series.unique():
        try:
            times[t] = re.findall('\d+:\d+', t)[0]
        except:
            times[t] = np.nan
    return series.map(times)

In [11]:
data['Sighting Time'] = time_picker(data['Occurred']) #find original sighting times
data['Report Time'] = time_picker(data['Reported']) #fight times that sightings were reported

In [12]:
data.head()

,Occurred,Reported,Posted,Location,Shape,Duration,Report,Sighting Time,Report Time
0,6/24/1996 00:30,6/26/1996 12:42,11/2/1999,"Aurora, CO",changed,1 hour,Referred to Marilyn Childs to call CO state di...,00:30,12:42
1,unknown,3/21/2006 12:11:10 PM 12:11,5/15/2006,"Bristol, PA",Light,About 1 min.,"Hi, I am an avid sky watcher and usually I not...",NaN,12:11
2,00:45,5/16/2015 12:34:36 AM 00:34,5/22/2015,"Saskatoon (Canada), SK",Light,2 minutes,We seemed to know to look up and and for me it...,00:45,12:34
3,01:00,10/21/2001 11:50:39 AM 11:50,11/20/2001,Dublin (Republic of Ireland),Light,a few seconds,"A ball of light, lighting up my pitch dark roo...",01:00,11:50
4,01:00,9/2/1999 09:52,2/5/2003,"Careywood & Bayview (Pend Oreille Lake), ID",Light,few minutes,"Ball of light similiar to reported ""Earht Ligh...",01:00,09:52


In [13]:
def datetime_conversion(series):
    #MAP DATETIMES FROM UNIQUE VALUES
    datetimes = {t:pd.to_datetime(t, errors='coerce') for t in series.unique()}
    return series.map(datetimes)

In [14]:
for col in ['Occurred','Reported','Posted']:
    data[col] = datetime_conversion(data[col])

In [15]:
data.head()

,Occurred,Reported,Posted,Location,Shape,Duration,Report,Sighting Time,Report Time
0,1996-06-24 00:30:00,1996-06-26 12:42:00,1999-11-02,"Aurora, CO",changed,1 hour,Referred to Marilyn Childs to call CO state di...,00:30,12:42
1,NaT,2006-03-21 12:11:00,2006-05-15,"Bristol, PA",Light,About 1 min.,"Hi, I am an avid sky watcher and usually I not...",NaN,12:11
2,2018-07-22 00:45:00,2015-05-16 00:34:00,2015-05-22,"Saskatoon (Canada), SK",Light,2 minutes,We seemed to know to look up and and for me it...,00:45,12:34
3,2018-07-22 01:00:00,2001-10-21 11:50:00,2001-11-20,Dublin (Republic of Ireland),Light,a few seconds,"A ball of light, lighting up my pitch dark roo...",01:00,11:50
4,2018-07-22 01:00:00,1999-09-02 09:52:00,2003-02-05,"Careywood & Bayview (Pend Oreille Lake), ID",Light,few minutes,"Ball of light similiar to reported ""Earht Ligh...",01:00,09:52


## Cleaning messy location data

In [16]:
data['Location'].unique()

array(['Aurora, CO', 'Bristol, PA', 'Saskatoon (Canada), SK', ...,
       'Wilhelmshaven (Germany)', 'Harpster, OH',
       'Oklahoma City (SW part), OK'], dtype=object)

In [17]:
def location_cleaner(loc):
    #CLEAN UP LOCATION STRINGS AND VAGUE TERMS
    descriptors = ['north of','south of','east of','west of',
                   'outside of','between','above','near','SW part',
                   'NW part','SE part','NE part']
    for r in ['?',', .']:
        loc = loc.replace(r,'')  
    if ('(' not in loc) or (')' not in loc):
        return loc
    else:
        pattern = '\(.*?\)'
        items = re.findall(pattern, loc)
        for i, item in enumerate(items):
            if item == '()':
                loc = loc.replace(item,'')
            if (len(item.split()) >= 3) or any(s in item for s in descriptors):
                loc = loc.replace(item,'')
        return re.sub(' +',' ',loc.replace(' ,',','))

def location_mapper(series):
    locations = {x:location_cleaner(x) for x in series.unique()} 
    return series.map(locations)

In [18]:
data['Location'] = location_mapper(data['Location'])

## Geocoding for latitude and longitude coordinates

In [19]:
len(data.Location) #109,496 rows

109496

In [20]:
data['Location'].nunique() #only 28,052 unique locations... Let's be efficient about this!

28052

In [21]:
'''
#GRABBING LAT-LON COORDINATES THROUGH GEOPY
from time import sleep
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim

geolocator = Nominatim()
locations = data['Location'].unique()
location_dict = {}

lines = 0
txt = os.getcwd() + '/data/lat-lon-coordinates.txt'
out_file = open(txt, 'w')
for idx, loc in enumerate(locations): 
    sleep(1.0) #Sleep for 1 second between each query.
    if idx == 0: out_file.write('Index Location Latitude Longitude')
    try:
        geoLoc = geolocator.geocode(loc, timeout = 60)
        out_file.write('\n%s|%s|%s|%s' % (idx, loc, geoLoc.latitude, geoLoc.longitude))
    except:
        out_file.write('\n%s|%s|%s|%s' % (idx, loc, 'NaN', 'NaN'))
    lines += 1
    print('{0}\r'.format(lines), end='')
out_file.close()
''';

In [22]:
df = pd.read_csv(os.getcwd() + '/data/lat-lon-coordinates.txt', sep='|', index_col=0, encoding='ISO-8859-1')
lat_dict = dict(zip(df['Location'],df['Latitude']))
lon_dict = dict(zip(df['Location'],df['Longitude']))

In [23]:
data['Latitude'] = data['Location'].map(lat_dict)
data['Longitude'] = data['Location'].map(lon_dict)
data = data.replace({'NaN':np.nan}, regex=True)

In [24]:
data.head()

,Occurred,Reported,Posted,Location,Shape,Duration,Report,Sighting Time,Report Time,Latitude,Longitude
0,1996-06-24 00:30:00,1996-06-26 12:42:00,1999-11-02,"Aurora, CO",changed,1 hour,Referred to Marilyn Childs to call CO state di...,00:30,12:42,39.729432,-104.831920
1,NaT,2006-03-21 12:11:00,2006-05-15,"Bristol, PA",Light,About 1 min.,"Hi, I am an avid sky watcher and usually I not...",NaN,12:11,40.100666,-74.851831
2,2018-07-22 00:45:00,2015-05-16 00:34:00,2015-05-22,"Saskatoon (Canada), SK",Light,2 minutes,We seemed to know to look up and and for me it...,00:45,12:34,52.157331,-106.529930
3,2018-07-22 01:00:00,2001-10-21 11:50:00,2001-11-20,Dublin,Light,a few seconds,"A ball of light, lighting up my pitch dark roo...",01:00,11:50,53.349764,-6.260273
4,2018-07-22 01:00:00,1999-09-02 09:52:00,2003-02-05,"Careywood & Bayview, ID",Light,few minutes,"Ball of light similiar to reported ""Earht Ligh...",01:00,09:52,NaN,NaN


## Cleaning up messy duration data

In [25]:
def duration_cleaner(s):
    #convert duration strings into minutes
    s = s.replace('few', '3.5').replace('1/2','0.5') #use convention that a few translates to 2 - 5 seconds.
    numericList = ["(\d+\.\d+)","(\d+)"]
    unitList = ["se[cs]", "secon[ds]","mi[ns]", "mi[mn]ut[es]","h[rs]", "hou[rs]"]
    unitConversion = {'s': 1/60, 'm': 1, 'h': 60, '6': 60}
    try:
        match_1 = re.search(re.compile("|".join(str(x) for x in numericList)), s).group(0)
        match_2 = re.search(re.compile("|".join(str(x) for x in unitList)), s.lower()).group(0)[0]
        s = float(match_1) * unitConversion[match_2]
    except:
        s = np.nan
    return s

In [26]:
data['Duration'][90]

'2-3 min'

In [27]:
duration_cleaner(data['Duration'][90])

2.0

In [28]:
data['Duration'] = data['Duration'].apply(duration_cleaner)

## Cleaning up messy shape categories

In [29]:
def shape_capitalizer(series):
    shapes = {x:x[0].upper()+x[1:].lower() for x in series.unique()}
    return series.map(shapes)

In [30]:
data['Shape'] = shape_capitalizer(data['Shape'])

In [31]:
data['Shape'].value_counts()

Light         23089
Circle        11559
Triangle      10538
Fireball       8660
Unknown        7879
Other          7790
Sphere         7537
Disk           6874
Oval           5038
Formation      3547
Changing       2751
Cigar          2723
Flash          2015
Rectangle      1882
Cylinder       1749
Diamond        1666
Chevron        1335
Egg            1018
Teardrop       1016
Cone            468
Cross           345
Delta             7
Round             2
Crescent          2
Hexagon           1
Pyramid           1
Changed           1
Triangular        1
Flare             1
Dome              1
Name: Shape, dtype: int64

In [32]:
data['Shape'] = data['Shape'].replace({'Changed':'Changing',
                                       'Flare':'Fireball',
                                       'Triangular':'Triangle'}, regex=True)

There are a lot of Unknown and Other shape categories, let's try to glean some shapes from the report descriptions.

In [33]:
def shape_category_counter(report, shape_categories):
    report = report.lower().split()
    cat_count = dict(zip(list(shape_categories), [0] * len(shape_categories)))
    for word in report:
        if word in shape_categories:
            cat_count[word] += 1
    cat_count = {k[0].upper()+k[1:]:v for k, v in cat_count.items() if v}
    return list(cat_count.keys())

In [34]:
shape_categories = [i.lower() for i in data['Shape'].value_counts().index if i not in ['Unknown','Other']]+['orb']

In [35]:
data['Shape Categories'] = data['Report'].apply(lambda x: shape_category_counter(x, shape_categories))

In [36]:
z = []
for x, y in zip(data['Shape'], data['Shape Categories']):
    if (x in ['Unknown','Other']) and (y != []):
        z.append(y)
    elif (x in ['Unknown','Other']) and (y == []):
        z.append(y + [x])
    elif (x not in ['Unknown','Other']) and (x not in y):
        z.append(y + [x])
    else:
        z.append(y)
data['Shape Categories Revised'] = z

In [37]:
del data['Shape Categories']

In [38]:
data.to_pickle(os.getcwd() + '/data/cleaned_reports_07202018.pkl') #end result, not perfect but much better